In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
path = '/home/kunal/Downloads/lord_of_machines/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
campaign = pd.read_csv(path +'campaign_data.csv')

In [ ]:
#weekday_train = pd.read_csv(path + 'weekday_train.csv')
#weekday_test = pd.read_csv(path + 'weekday_test.csv')

In [ ]:
train_input = pd.read_csv(path + 'impact_encoded_train.csv')
test_input = pd.read_csv(path + 'impact_encoded_test.csv')

In [ ]:
#train_input1 = pd.read_csv(path + 'train_new.csv')
#test_input1 = pd.read_csv(path + 'test_new.csv')

### Generate user profile based on their interests in communication type

In [ ]:
train = train.merge(campaign, on = 'campaign_id',how = 'left')
test = test.merge(campaign, on = 'campaign_id',how = 'left')

In [ ]:
data = pd.concat([train[['user_id','communication_type']],test[['user_id','communication_type']]])

In [ ]:
user_profile = pd.crosstab(data['user_id'],data['communication_type'])

In [ ]:
user_profile.reset_index(inplace = True)

In [ ]:
train_input = train_input.merge(user_profile,on = 'user_id',how = 'left')
test_input = test_input.merge(user_profile,on = 'user_id',how = 'left')

In [ ]:
#train_input = pd.concat([train_input,weekday_train],axis = 1)
#test_input = pd.concat([test_input,weekday_test],axis = 1)

In [ ]:
#train_input['impact_encoded_weekday'] = train['impact_encoded_weekday']
#train_input['impact_encoded_open_weekday'] = train['impact_encoded_open_weekday']
#test_input['impact_encoded_weekday'] = test['impact_encoded_weekday']
#test_input['impact_encoded_open_weekday'] = test['impact_encoded_open_weekday']

In [ ]:
#weekday_train.to_csv(path + 'weekday_train.csv',index = False)
#weekday_test.to_csv(path + 'weekday_test.csv',index = False)

In [ ]:
train_input.columns

In [ ]:
user_features  = pd.read_csv(path + 'user_features.csv')

In [ ]:
#train_input['user_id'] = train['user_id']
#test_input['user_id'] = test['user_id']
#train_input['is_open'] = train['is_open']
#train_input['is_click'] = train['is_click']

In [ ]:
#campaign_features = pd.read_csv(path + '2_gram_campaign_features.csv')

In [ ]:
#campaign_features.head()

## Modelling part

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(50,random_state = 10)
user_features_matrix = pca.fit_transform(user_features.iloc[:,1:])

In [ ]:
user_features_matrix = pd.DataFrame(user_features_matrix)
user_features_matrix['user_id'] = user_features['user_id']
train_input = train_input.merge(user_features_matrix,on = 'user_id',how = 'left')

In [ ]:
#pca2 = PCA(30,random_state = 10)
#cluster_feature = pca2.fit_transform(campaign_features.iloc[:,1:])

In [ ]:
#cluster_feature = pd.DataFrame(cluster_feature)
#cluster_feature['campaign_id'] = campaign['campaign_id']

In [ ]:
#train_input['campaign_id'] = train['campaign_id']
#train_input = train_input.merge(campaign_features,on = 'campaign_id',how = 'left')

In [ ]:
test_input = test_input.merge(user_features_matrix,on = 'user_id',how = 'left')
#test_input['campaign_id'] = test['campaign_id']
#test_input = test_input.merge(campaign_features,on = 'campaign_id',how = 'left')

In [ ]:
#x = encoded_features(train_input,train_input)
#y = encoded_features(df = train_input,df_new=test_input)
x = train_input.copy()
y = test_input.copy()
count_feature = pd.concat([train['user_id'],test['user_id']])
count_feature = count_feature.value_counts()
x['counts'] = x['user_id'].map(count_feature)
y['counts'] = y['user_id'].map(count_feature)
x = x.drop(['user_id','is_click','is_open','campaign_id','id','communication_type'],axis = 1)
y = y.drop(['user_id','campaign_id','id','communication_type'],axis = 1)
#x['total_open_percentage'] = x['total_open']/x['counts']
#y['total_open_percentage'] = y['total_open']/y['counts']

In [ ]:
### Check the columns in X and Y
### They should be same and id variables should not be present
print(x.columns)
#print('*-'*50)
print(y.columns)

In [ ]:
#x.drop(['id','communication_type'],axis = 1,inplace = True)
#y.drop(['id','communication_type'],axis = 1,inplace = True)

In [ ]:
#x.drop(['impact_encoded_weekday','impact_encoded_open_weekday'],axis = 1,inplace = True)
#y.drop(['impact_encoded_weekday','impact_encoded_open_weekday'],axis = 1,inplace = True)

In [ ]:
#from sklearn.model_selection import train_test_split
#Xtrain,Xtest,ytrain,ytest = train_test_split(x,train['is_click'],test_size = 0.7)

In [ ]:
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.linear_model import LogisticRegression

xg = XGBClassifier(n_estimators = 500,max_depth= 7,gamma = 20,colsample_bylevel=0.9,colsample_bytree=0.9)
#cb = CatBoostClassifier()
#knn = KNeighborsClassifier()
#rf = RandomForestClassifier()
#et = ExtraTreesClassifier()
#lr = LogisticRegression()

In [ ]:
### Choose your algo and use fit method (replace classifier by the name of your algo constructor)
xg.fit(x,train['is_click'])


In [ ]:
predictions = xg.predict_proba(y)

In [ ]:
sample = pd.read_csv(path + 'sample.csv')

In [ ]:

name_of_submission_file = 'final_sub2'    # select name of file
sample['is_click'] = predictions[:,1]
sample.to_csv(path + '{}.csv'.format(name_of_submission_file),index = False)

In [ ]:
from xgboost import plot_importance

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(train['is_click'],xg.predict_proba(x)[:,1]))

In [ ]:
plot_importance(xg)